In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow, get_cmap
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 10
TITLE = "15-16 February 2024"
SUBTITLE = "8 AM 17 February 2024"
SETPOINT_LOCS = {}
sts = utc(2024, 2, 16, 0)
ets = utc(2024, 2, 17, 18)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
df = df[~df['nwsli'].isin(['DSXI4', 'DMX'])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df[df["state"] == 'IA'])
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(
        lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df[df["state"] == 'IA'])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,WY,CYS,46.0,-106.3500,41.3800,POINT (-529418.416 -382020.879),True,0,True,LSR,-26,5
1,WY,CYS,28.5,-106.9700,41.1500,POINT (-583052.996 -403334.046),True,1,True,LSR,-28,4
2,WY,CYS,24.0,-106.9400,41.0700,POINT (-581273.154 -412414.766),True,2,True,LSR,-28,4
3,WY,CYS,22.5,-106.7100,41.0600,POINT (-562150.128 -415119.858),True,3,True,LSR,-27,4
4,WY,CYS,22.5,-106.2300,41.3600,POINT (-519600.362 -385004.234),True,4,True,LSR,-25,5
...,...,...,...,...,...,...,...,...,...,...,...,...
5901,OK,OUN,0.0,-96.7781,34.1133,POINT (297869.423 -1203083.708),True,MADO2,True,COOP,1,-22
5902,OK,OUN,0.0,-97.7333,36.8000,POINT (202308.251 -908291.377),True,MEDO2,True,COOP,-1,-12
5903,OK,OUN,0.0,-97.9833,34.6500,POINT (185208.022 -1147102.167),True,MLWO2,True,COOP,-2,-20
5904,OK,OUN,0.0,-99.4956,35.1464,POINT (46030.921 -1094182.969),True,MRVO2,True,COOP,-7,-18


In [21]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'IA-CN-5',
        'IA-CN-22',
        'BLLI4',
        'IA-TM-4',
        'IA-CN-17',
        'IA-JC-5',
        'SGYI4',
        'IA-LC-12',
        'IA-FM-1',
        'ROKI4',
        'GRTM7',
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        ('IA-PC-10', 0.9),
        ('EMMI4', 0.5),
        ('IA-TY-2', 0.5),
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=1)
    # draw_setpoints(mp)

    #plotsqw(mp)
    res = mp.postprocess(filename='240217.png')
    mp.close()

main()

     state  wfo  val       lon       lat                             geo  \
925     IA  DMX  4.2 -93.66000  41.53000  POINT (527343.383 -365448.323)   
1063    IA  DMX  4.0 -93.64000  41.60000  POINT (528419.852 -357555.888)   
1178    IA  DMX  3.9 -93.86000  41.61000  POINT (510103.755 -357836.698)   
1379    IA  DMX  3.5 -93.64000  41.58000  POINT (528586.469 -359774.167)   
1423    IA  DMX  3.4 -93.98000  41.60000  POINT (500235.556 -359683.633)   
...    ...  ...  ...       ...       ...                             ...   
5866    IA  ARX  0.0 -91.09590  42.78573  POINT (724974.993 -206672.548)   
5867    IA  ARX  0.0 -91.83025  42.88262  POINT (664388.846 -202188.022)   
5868    IA  ARX  0.0 -92.44330  43.24190  POINT (611089.758 -167166.650)   
5869    IA  ARX  0.0 -91.45370  42.77540  POINT (696095.426 -210928.088)   
5870    IA  ARX  0.0 -91.79520  43.30430  POINT (662602.115 -155190.678)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
925                True  